In [1]:
import sys
import scanpy as sc
from sklearn.preprocessing import OneHotEncoder
# cd to mvTCR directory
sys.path.append('/path/to/mvTCR-master')
from tcr_embedding.models.model_selection import run_model_selection
from tcr_embedding.utils_preprocessing import encode_tcr
import tcr_embedding.utils_training as utils
from tcr_embedding.utils_preprocessing import group_shuffle_split

/nfs/team205/lisa_dinithi/miniconda3/envs/mvTCR/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata = sc.read_h5ad('/23-01-07-mvtcr-prep/mvtcr-input-scirpy.h5ad')
adata

AnnData object with n_obs × n_vars = 17157 × 33538
    obs: 'file', 'donor', 'anno_lvl_2_final_clean', 'multi_chain', 'extra_chains', 'IR_VJ_1_c_call', 'IR_VJ_2_c_call', 'IR_VDJ_1_c_call', 'IR_VDJ_2_c_call', 'IR_VJ_1_consensus_count', 'IR_VJ_2_consensus_count', 'IR_VDJ_1_consensus_count', 'IR_VDJ_2_consensus_count', 'IR_VJ_1_d_call', 'IR_VJ_2_d_call', 'IR_VDJ_1_d_call', 'IR_VDJ_2_d_call', 'IR_VJ_1_duplicate_count', 'IR_VJ_2_duplicate_count', 'IR_VDJ_1_duplicate_count', 'IR_VDJ_2_duplicate_count', 'IR_VJ_1_j_call', 'IR_VJ_2_j_call', 'IR_VDJ_1_j_call', 'IR_VDJ_2_j_call', 'IR_VJ_1_junction', 'IR_VJ_2_junction', 'IR_VDJ_1_junction', 'IR_VDJ_2_junction', 'IR_VJ_1_junction_aa', 'IR_VJ_2_junction_aa', 'IR_VDJ_1_junction_aa', 'IR_VDJ_2_junction_aa', 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_productive', 'IR_VJ_2_productive', 'IR_VDJ_1_productive', 'IR_VDJ_2_productive', 'IR_VJ_1_v_call', 'IR_VJ_2_v_call', 'IR_VDJ_1_v_call', 'IR_VDJ_2_v_call', 'has_ir', 'rec

In [12]:
# pass requried TCR info for mvTCR

len_beta = adata.obs['IR_VJ_1_junction_aa'].str.len().max()
len_alpha= adata.obs['IR_VDJ_1_junction_aa'].str.len().max()
pad = max(len_beta, len_alpha)

encode_tcr(adata, 'IR_VJ_1_junction_aa', 'IR_VDJ_1_junction_aa', pad)

# one hot encode the conditional variable (donor) and save to obsm
enc = OneHotEncoder(sparse=False)
enc.fit(adata.obs['donor'].to_numpy().reshape(-1, 1))
adata.obsm['donor'] = enc.transform(adata.obs['donor'].to_numpy().reshape(-1, 1))
adata.uns['donor_enc'] = enc.categories_

# set training and validation split
train, val = group_shuffle_split(adata, group_col='clonotype', val_split=0.20, random_seed=123456789)
adata.obs['set'] = 'train'
adata.obs.loc[val.obs.index, 'set'] = 'val'

Modify the weight ratio used in the loss function by changing contrib_GEX and contrib_VDJ in the code below, and give each new run a new name.

In [13]:
# parameters
name = 'logger_test_2'
general_savedir = 'models/'
n_epochs = 200
timeout = 24*60*60 # seconds
n_samples = 15
n_gpus = 3

contrib_GEX = 1
contrib_VDJ = 1

params_experiment = {
    'study_name': name,
    'comet_workspace': None, 
    'model_name': 'moe',
    'balanced_sampling': 'clonotype',
    'metadata': [],
    'save_path': general_savedir+name,
    'conditional': 'donor', # one hot encoded donor_id
    'n_epochs': n_epochs,
}

params_optimization = {
    'name': 'pseudo_metric',
    'prediction_labels':
        {'clonotype': contrib_VDJ, # VDJ clone_id
         'anno_lvl_2_final_clean' : contrib_GEX} # GEX label
}

run_model_selection(
    adata, 
    params_experiment, 
    params_optimization, 
    n_samples, 
    timeout, 
    n_gpus) 

[I 2023-01-16 16:28:38,317] A new study created in RDB with name: logger_test_2
  0%|                                                     | 0/4 [00:00<?, ?it/s]computing neighbors
    using data matrix X directly
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:06)
computing neighbors
    using data matrix X directly
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
 25%|███████████▎                                 | 1/4 [00:20<01:01, 20.37s/it]computing neighbors
    using data matrix X directly
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency ma

Study statistics:
  Number of finished trials: 2
  Number of pruned trials: 0
  Number of complete trials: 2
Best trial: 
  trial_0
  Value: 0.6324090034424144
